# OpenLane 模型评估与可视化分析

In [ ]:
import sys
import os
from pathlib import Path
from unlanedet.evaluation.openlane_evaluator import OpenLaneEvaluator
from omegaconf import OmegaConf

iou_threshold  = 0.5
width = 30
param_config = OmegaConf.create()
param_config.data_root = "/data1/lxy_log/workspace/ms/OpenLane/dataset/raw/lane3d_300/test"
param_config.output_dir = "/data1/lxy_log/workspace/ms/UnLanedet/output/llanet/mobilenetv4_small_gsafpn_openlane/test"
param_config.parse_evaluate_result_script = "/data1/lxy_log/workspace/ms/UnLanedet/tools/read_open2d_csv_results.py"
param_config.generate_visualization = True

open_lane_evaluator = OpenLaneEvaluator(
    cfg=param_config, 
    evaluate_bin_path="/data1/lxy_log/workspace/ms/UnLanedet/tools/exe/openlane_2d_evaluate", 
    iou_threshold=iou_threshold, 
    width=width
)

## 1. 检查评估文件结构

In [ ]:
def check_setup():
    print('=== 评估环境检查 ===')
    validation_dir = EVAL_RESULTS_DIR / 'validation'
    if validation_dir.exists():
        json_count = len(list(validation_dir.glob('*.json')))
        print(f'✓ 验证结果目录: {validation_dir} ({json_count} 个JSON文件)')
    else:
        print(f'✗ 验证结果目录: {validation_dir} (不存在)')
check_setup()

## 2. 运行官方评估工具

In [ ]:
def run_evaluation():
    print('=== 运行官方评估工具 ===')
    validation_dir = EVAL_RESULTS_DIR / 'validation'
    if not validation_dir.exists():
        print('错误: 验证结果目录不存在')
        print('请先运行模型推理生成预测结果!')
        return False
    test_list_path = EVAL_RESULTS_DIR / 'test_list.txt'
    with open(test_list_path, 'w') as f:
        for json_file in validation_dir.rglob('*.json'):
            try:
                with open(json_file, 'r') as jf:
                    data = json.load(jf)
                    file_path = data.get('file_path', '')
                    if file_path and file_path.startswith('validation/'):
                        f.write(file_path.replace('.json', '.jpg') + '\n')
            except Exception as e:
                print(f'警告: 无法读取 {json_file}: {e}')
    print(f'测试列表已生成: {test_list_path}')
    print('评估工具需要手动运行或配置后执行')
    return True
run_evaluation()

## 3. 分析评估结果

In [ ]:
def analyze_results():
    csv_dir = EVAL_RESULTS_DIR / 'csv_results'
    if not csv_dir.exists():
        print('CSV结果目录不存在，请先运行官方评估工具!')
        return
    print('CSV结果目录存在:', csv_dir)
    csv_files = list(csv_dir.glob('*.csv'))
    print(f'找到 {len(csv_files)} 个CSV文件')
    for file in csv_files:
        size = file.stat().st_size / 1024
        print(f'  - {file.name} ({size:.1f} KB)')
analyze_results()

## 4. 生成可视化

In [ ]:
def generate_viz():
    VISUALIZATION_DIR.mkdir(parents=True, exist_ok=True)
    print(f'可视化目录: {VISUALIZATION_DIR}')
    print('可使用 tools/read_open2d_csv_results.py 生成详细可视化')
generate_viz()